In [8]:
import pandas as pd
import os

# Excelファイルが保存されているディレクトリのパスを指定
directory_path = 'C:\\bankdata\\tousan\\tantai'

# ディレクトリ内のすべてのExcelファイルをループで処理
for filename in os.listdir(directory_path):
    if filename.endswith(".xlsx") or filename.endswith(".xls"):
        file_path = os.path.join(directory_path, filename)
        excel_data = pd.ExcelFile(file_path)
        
        # すべてのシートのデータを結合するための空のリストを作成
        df_list = []
        
        # 各シートのデータをループして結合
        for sheet_name in excel_data.sheet_names:
            # シートのデータを読み込む
            df = pd.read_excel(excel_data, sheet_name=sheet_name)
            # シート名の列を追加（元のシートを特定するため）
            df['SheetName'] = sheet_name
            # リストに追加
            df_list.append(df)
        
        # すべてのデータフレームを結合
        combined_df = pd.concat(df_list, ignore_index=True)
        
        # 保存するCSVファイルのパス（拡張子を.csvに変更）
        csv_filename = filename.replace('.xlsx', '.csv').replace('.xls', '.csv')
        csv_file_path = os.path.join(directory_path, csv_filename)
        
        # 結合したデータをCSVファイルとして保存（エンコーディング指定）
        combined_df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')  # UTF-8で保存、Excel互換のBOM付き
        
        print(f"{filename} のシートを結合して {csv_filename} に保存しました。")


In [10]:
import os
import re
from collections import defaultdict

# フォルダ定義
folders = {
    '倒産かつ連結': 'C:/bankdata/tousan/renketsu/',
    '倒産かつ単体': 'C:/bankdata/tousan/tantai/',
    '非倒産かつ連結': 'C:/bankdata/hitousan/renketsu/',
    '非倒産かつ単体': 'C:/bankdata/hitousan/tantai/'
}

# 証券番号の出現記録
code_map = defaultdict(list)

# 各ファイルの証券番号を抽出して記録
for category, folder_path in folders.items():
    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            match = re.match(r'(E?\d{4,5})', filename)
            if match:
                code = match.group(1)
                full_path = os.path.join(folder_path, filename)
                code_map[code].append(full_path)

# 重複チェック
print("\n=== 証券番号の重複チェック結果 ===")
duplicate_found = False
for code, paths in code_map.items():
    if len(paths) > 1:
        duplicate_found = True
        print(f"\n⚠ 重複: 証券番号 {code} が {len(paths)} 件見つかりました:")
        for path in paths:
            print(f" - {path}")

if not duplicate_found:
    print("✅ 重複は検出されませんでした。")


=== 証券番号の重複チェック結果 ===

⚠ 重複: 証券番号 2228 が 2 件見つかりました:
 - C:/bankdata/tousan/tantai/2228_901_zaimu_-20180831_全体.csv
 - C:/bankdata/hitousan/tantai/2228_901_zaimu_-20150831_全体.csv

⚠ 重複: 証券番号 E04725 が 2 件見つかりました:
 - C:/bankdata/tousan/tantai/E04725_901_zaimu_-20160331_全体.csv
 - C:/bankdata/hitousan/tantai/E04725_xxxx_zaimu_901_-20120331.csv
